In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns
import arviz as az
from sklearn import datasets
from statsmodels.tsa import stattools
import statsmodels.api as sm
import pymc3 as pm
import pymc

import sys
sys.path.insert(0, '..')
from utils.plot_lib import set_default

set_default(figsize=(6, 4))

### Numerical stability

In [2]:
from scipy.stats import norm

## Consider a mixture of two normal distributions with equal weights (w1 = w2 = 1/2)
## Component 1 has mean 0 and standard deviation 1
## Component 2 has mean 1 and standard deviation 1
## The observation is x = 50
## What is Pr(c = 1 | x)?

mu1 = 0
mu2 = 1
sig1 = 1
sig2 = 1
x = 50

k1 = norm(loc = mu1, scale = sig1)
k2 = norm(loc = mu2, scale = sig2)

print('The probability that observation x = 50 belongs to component 1 is: {}'.format(k1.pdf(x)))
print('The probability that observation x = 50 belongs to component 2 is: {}'.format(k2.pdf(x)))
print('Computing Ratio. Numeral stability!: {}'.format(k1.pdf(x) / (k1.pdf(x) + k2.pdf(x))))

The probability that observation x = 50 belongs to component 1 is: 0.0
The probability that observation x = 50 belongs to component 2 is: 0.0
Computing Ratio. Numeral stability!: nan


In [3]:
## What if x=3?  Two ways to do the calculation
## One way:  Direct calculation

x = 3
z1 = k1.pdf(3)
z2 = k2.pdf(3)
print(z1 / (z1+z2))

0.07585818002124355


In [4]:
## A second way:  Compute in the logarithm scale, add b 
## to all values, and then exponentiate before standardizing

## A second way:  Compute in the logarithm scale, add b 
## to all values, and then exponentiate before standardizing
lz1 = k1.logpdf(x)
lz2 = k2.logpdf(x)
b = 3
print(np.exp(lz1 - b) / (np.exp(lz1 - b) + np.exp(lz2 - b)))

0.07585818002124356


In [5]:
# Other ways to compute it
x = 50
lz1 = k1.logpdf(x)
lz2 = k2.logpdf(x)
b = np.max([lz1, lz2])
print(np.exp(lz1 - b) / (np.exp(lz1 - b) + np.exp(lz2 - b)))

3.1799709001977496e-22


In [6]:
## Also right (just more cumbersome)
lz1 = -0.5 * np.log(2 * np.pi) - 0.5 * x**2
lz2 = -0.5 * np.log(2 * np.pi) - 0.5 * (x - mu2)**2
b = np.max([lz1, lz2])
print(np.exp(lz1 - b) / (np.exp(lz1 - b) + np.exp(lz2 - b)))

3.1799709001977496e-22
